In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
dt = pd.read_csv('../task1/Dataset .csv')
pd.set_option('display.max_columns', None)
dt

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",1500,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",1500,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,208,��stanbul,"Kemanke�� Karamustafa Pa��a Mahallesi, R۱ht۱m ...",Karak�_y,"Karak�_y, ��stanbul",28.977392,41.022793,Turkish,80,Turkish Lira(TL),No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz A��ac۱,208,��stanbul,"Ko��uyolu Mahallesi, Muhittin ��st�_nda�� Cadd...",Ko��uyolu,"Ko��uyolu, ��stanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",105,Turkish Lira(TL),No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.034640,41.055817,"Italian, World Cuisine",170,Turkish Lira(TL),No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,A���k Kahve,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.036019,41.057979,Restaurant Cafe,120,Turkish Lira(TL),No,No,No,No,4,4.0,Green,Very Good,901


In [3]:
enc_dt = dt
enc = enc_dt.drop_duplicates(subset=['Restaurant Name'],keep='first')
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(dt[['City','Cuisines', 'Rating text',]])

In [4]:
feature_matrix = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out())

In [5]:
similarity_matrix = cosine_similarity(feature_matrix)

In [25]:
similarity_df = pd.DataFrame(similarity_matrix, index=enc_dt['Restaurant Name'], columns=enc_dt['Restaurant Name'])

In [6]:
def recommend_restaurants(restaurant_name, num_recommendations=5):
    if not isinstance(restaurant_name, list):
        print("Restaurant not found!")
        return 0
    else:
        return similarity_df[restaurant_name].sum(axis=1).sort_values(ascending=False).iloc[1:num_recommendations+1].index.tolist()

Creating resturent to resturent distance matric

In [7]:
res_r = dt[['Restaurant Name','Longitude','Latitude']].drop_duplicates(subset=['Restaurant Name'],keep='first').set_index('Restaurant Name')
x2 = pd.DataFrame.to_numpy(dt[['Restaurant Name','Longitude']].drop_duplicates(subset=['Restaurant Name'],keep='first').set_index('Restaurant Name'))
y2 = pd.DataFrame.to_numpy(dt[['Restaurant Name','Latitude']].drop_duplicates(subset=['Restaurant Name'],keep='first').set_index('Restaurant Name'))

In [8]:
res_c = dt[['Restaurant Name','Longitude','Latitude']].drop_duplicates(subset=['Restaurant Name'],keep='first').set_index('Restaurant Name').transpose()
x1 = pd.DataFrame.to_numpy(dt[['Restaurant Name','Longitude']].drop_duplicates(subset=['Restaurant Name'],keep='first').set_index('Restaurant Name').transpose())
y1 = pd.DataFrame.to_numpy(dt[['Restaurant Name','Latitude']].drop_duplicates(subset=['Restaurant Name'],keep='first').set_index('Restaurant Name').transpose())

In [9]:
#creating 7446 x 7446 matrix
res_mat = pd.DataFrame([[0 for _ in range(7446)] for _ in range(7446)],index = dt['Restaurant Name'].drop_duplicates(keep='first'),columns=res_c.columns)
mat = pd.DataFrame.to_numpy(res_mat)


In [10]:
mat_x1 = pd.DataFrame.to_numpy(res_mat).astype(float)
mat_x1[:] = x1[0]

In [11]:
mat_x2 = pd.DataFrame.to_numpy(res_mat).astype(float)
mat_x2 = np.tile(x2, (1, 7446))

In [12]:
mat_y1 = pd.DataFrame.to_numpy(res_mat).astype(float)
mat_y1[:] = y1[0]

In [13]:
mat_y2 = pd.DataFrame.to_numpy(res_mat).astype(float)
mat_y2 = np.tile(y2, (1, 7446))

In [14]:
sqr_dl_x = (mat_x2 - mat_x1)**2
sqr_dl_x

array([[0.00000000e+00, 1.80472356e-04, 8.58255616e-04, ...,
        8.46269273e+03, 8.46243902e+03, 8.46427949e+03],
       [1.80472356e-04, 0.00000000e+00, 1.82585290e-03, ...,
        8.46022124e+03, 8.45996757e+03, 8.46180778e+03],
       [8.58255616e-04, 1.82585290e-03, 0.00000000e+00, ...,
        8.46808363e+03, 8.46782984e+03, 8.46967090e+03],
       ...,
       [8.46269273e+03, 8.46022124e+03, 8.46808363e+03, ...,
        0.00000000e+00, 1.90161342e-06, 7.43730310e-05],
       [8.46243902e+03, 8.45996757e+03, 8.46782984e+03, ...,
        1.90161342e-06, 0.00000000e+00, 1.00059409e-04],
       [8.46427949e+03, 8.46180778e+03, 8.46967090e+03, ...,
        7.43730310e-05, 1.00059409e-04, 0.00000000e+00]])

In [15]:
sqr_dl_y = (mat_y2 - mat_y1)**2
sqr_dl_y

array([[0.00000000e+00, 1.37710225e-04, 2.54753521e-04, ...,
        7.01739923e+02, 7.01854464e+02, 6.97981137e+02],
       [1.37710225e-04, 0.00000000e+00, 7.67068416e-04, ...,
        7.02361789e+02, 7.02476381e+02, 6.98601336e+02],
       [2.54753521e-04, 7.67068416e-04, 0.00000000e+00, ...,
        7.00894552e+02, 7.01009024e+02, 6.97138033e+02],
       ...,
       [7.01739923e+02, 7.02361789e+02, 7.00894552e+02, ...,
        0.00000000e+00, 4.67359542e-06, 5.04689756e-03],
       [7.01854464e+02, 7.02476381e+02, 7.01009024e+02, ...,
        4.67359542e-06, 0.00000000e+00, 5.35873338e-03],
       [6.97981137e+02, 6.98601336e+02, 6.97138033e+02, ...,
        5.04689756e-03, 5.35873338e-03, 0.00000000e+00]])

In [16]:
dist_mat = np.sqrt(sqr_dl_x +sqr_dl_y)
dist_mat

array([[0.00000000e+00, 1.78376731e-02, 3.33617916e-02, ...,
        9.57310433e+01, 9.57303164e+01, 9.57196982e+01],
       [1.78376731e-02, 0.00000000e+00, 5.09207356e-02, ...,
        9.57213823e+01, 9.57206558e+01, 9.57100262e+01],
       [3.33617916e-02, 5.09207356e-02, 0.00000000e+00, ...,
        9.57547815e+01, 9.57540540e+01, 9.57434538e+01],
       ...,
       [9.57310433e+01, 9.57213823e+01, 9.57547815e+01, ...,
        0.00000000e+00, 2.56421700e-03, 7.15630533e-02],
       [9.57303164e+01, 9.57206558e+01, 9.57540540e+01, ...,
        2.56421700e-03, 0.00000000e+00, 7.38836436e-02],
       [9.57196982e+01, 9.57100262e+01, 9.57434538e+01, ...,
        7.15630533e-02, 7.38836436e-02, 0.00000000e+00]])

In [17]:
dl_matri = pd.DataFrame(dist_mat,index = dt['Restaurant Name'].drop_duplicates(keep='first'),columns=res_c.columns)

                                                                preferencesed based filtering

In [20]:
import random
print(random.choice(pd.unique(dt['Restaurant Name']).tolist()))

Love Desserts


In [18]:
user_preferences = {
    
    'Cuisines':['Seafood', 'Filipino, Asian', 'European'],
    'City': 'Pasay City',
    #'Price range': '',
    'Rating' : 2,
    
}
print(user_preferences)

{'Cuisines': ['Seafood', 'Filipino, Asian', 'European'], 'City': 'Pasay City', 'Rating': 2}


                                                                Choses Random fields from dataset

In [21]:
user_preferences = {
    
    'Cuisines': random.choice(pd.unique(dt['Cuisines']).tolist()), #['Seafood', 'Filipino, Asian', 'European'],
    'City': random.choice(pd.unique(dt['City']).tolist()),         #'Pasay City',
    'Rating' : 2,
    
} 
print(user_preferences)


{'Cuisines': 'Asian, Chinese, Vegetarian', 'City': 'Chandigarh', 'Rating': 2}


In [22]:
matching_restaurants = dt[
    (dt['City'] == user_preferences['City']) &
    (dt['Cuisines'].str.contains('|'.join(user_preferences['Cuisines']), case=False, na=False)) &
    (dt['Aggregate rating'] > user_preferences['Rating'])
]
view_res = matching_restaurants[['Restaurant ID','Restaurant Name','Address','Cuisines','Average Cost for two','Currency','Rating text','Aggregate rating',]]
view_res

,Restaurant ID,Restaurant Name,Address,Cuisines,Average Cost for two,Currency,Rating text,Aggregate rating
787,122003,The Night Factory,"Phase 1, Chandigarh Industrial Area, Chandigarh","North Indian, Chinese, Continental, Pizza",850,Indian Rupees(Rs.),Good,3.7
788,121552,Brooklyn Central,"Courtyard, Elante Mall, Phase 1, Chandigarh In...","American, Cafe",1600,Indian Rupees(Rs.),Very Good,4.2
789,121214,Chili's,"312 B, 3rd Floor, Elante Mall, Phase 1, Chandi...","Mexican, American, Italian",1400,Indian Rupees(Rs.),Very Good,4.3
790,121553,Kylin Experience,"312 A, 3rd Floor, Elante Mall, Phase 1, Chandi...","Japanese, Chinese, Thai, Malaysian, Burmese, A...",1600,Indian Rupees(Rs.),Very Good,4.1
791,121312,Mocha,"Ground Floor, Elante Mall, Phase 1, Chandigarh...","Continental, Italian, Thai, Finger Food",1400,Indian Rupees(Rs.),Very Good,4.0
792,121335,Pirates of Grill,"313, Third Floor, Elante Mall, Phase 1, Chandi...","North Indian, Continental, Asian",1200,Indian Rupees(Rs.),Very Good,4.0
793,120014,Barbeque Nation,"SCO 39, Madhya Marg, Sector 26, Chandigarh","North Indian, Chinese",1300,Indian Rupees(Rs.),Excellent,4.5
794,123125,TGI Friday's,"SCO 51, Madhya Marg, Sector 26, Chandigarh","American, Tex-Mex",1800,Indian Rupees(Rs.),Very Good,4.3
795,120221,Pal Dhaba,"SCO 151 & 152, Sector 28 D, Sector 28, Chandigarh",North Indian,700,Indian Rupees(Rs.),Good,3.8
796,122064,Midnight Chef,"SCO 329-332, Sector 35B, Sector 35, Chandigarh","North Indian, Chinese, Continental, Italian, B...",1000,Indian Rupees(Rs.),Average,3.4


                                Somtime you didn't get enough records becouse random values won't match, generate random values again 

                                                                        View hotels nearby 

In [32]:
flitered_res = view_res['Restaurant Name']
print("Nearby resturents from ",view_res['Restaurant Name'].iloc[1])
rnms = dl_matri[flitered_res.iloc[0]].sort_values()[1:].index
nby_res = dt[dt['Restaurant Name'].isin(rnms)].sort_values(by='Aggregate rating',ascending=False)[['Restaurant ID','Restaurant Name','Address','Cuisines','Average Cost for two','Currency','Rating text','Aggregate rating']]
nby_res.head(6)


Nearby resturents from  Brooklyn Central


,Restaurant ID,Restaurant Name,Address,Cuisines,Average Cost for two,Currency,Rating text,Aggregate rating
1381,18384115,Caterspoint,"S-27/11, DLF Phase 3, Gurgaon","Mexican, American, Healthy Food",500,Indian Rupees(Rs.),Excellent,4.9
589,18269368,AB's Absolute Barbecues,"Mezzanaine Floor, Centurion Star Tower, Deira ...","Continental, Indian",160,Emirati Diram(AED),Excellent,4.9
374,17580142,McGuire's Irish Pub & Brewery,"600 E Gregory Street, Pensacola, FL 32502","Burger, Bar Food, Steak",40,Dollar($),Excellent,4.9
9303,18217279,Miann,"57 Fort Street, Auckland Auckland CBD",Desserts,25,NewZealand($),Excellent,4.9
9299,7001086,Milse,"The Pavilions, 27 Tyler Street, Britomart, Auc...",Desserts,50,NewZealand($),Excellent,4.9
9296,7417450,Talaga Sampireun,"Jl. Boulevard Bintaro Jaya Blok B7/N1, Bintaro...","Sunda, Indonesian",200000,Indonesian Rupiah(IDR),Excellent,4.9


In [33]:
res_name = flitered_res.tolist()
rec_list = recommend_restaurants(res_name, num_recommendations=5)

print("Recommended Restaurants:", rec_list)

Recommended Restaurants: ['Aromas of Pind', 'Barbeque Nation', 'Barbeque Nation', 'Barbeque Nation', 'The Culinary Pitaara']


In [34]:
nby_res[ nby_res['Restaurant Name'].isin(rec_list) ]

,Restaurant ID,Restaurant Name,Address,Cuisines,Average Cost for two,Currency,Rating text,Aggregate rating
9262,2800856,Barbeque Nation,"1st Floor, ATR Towers, Harbour Park Road, Pan...","North Indian, Chinese, Mediterranean",1600,Indian Rupees(Rs.),Excellent,4.9
2409,25570,Barbeque Nation,"24, 1st Floor, Park Center Building, Park Stre...","North Indian, Chinese",1600,Indian Rupees(Rs.),Excellent,4.9
2414,20842,Barbeque Nation,"K1, RDB Boulevard, Block EP & GP, Sector 5, Sa...","North Indian, Chinese",1600,Indian Rupees(Rs.),Excellent,4.9
2536,3300958,Barbeque Nation,"2nd Floor, Eternity Mall, Amravati Road, Sitab...","North Indian, European, Mediterranean",1600,Indian Rupees(Rs.),Excellent,4.9
2289,2100702,Barbeque Nation,"2nd Floor, Adityam Building, Ulubari, Guwahati",North Indian,1500,Indian Rupees(Rs.),Excellent,4.9
2436,800089,Barbeque Nation,"3rd Floor, Riverside Mall, Vipin Khand, Gomti ...",North Indian,1600,Indian Rupees(Rs.),Excellent,4.7
768,2900633,Barbeque Nation,"Plot 12-13/557, 3rd Floor, Mauza Samantapuri,G...","Mediterranean, Asian, Continental, North India...",1200,Indian Rupees(Rs.),Excellent,4.6
2383,900969,Barbeque Nation,"6th Floor, Imperial Trade Centre, Mahatma Gand...","European, North Indian, Mediterranean, American",1500,Indian Rupees(Rs.),Excellent,4.5
587,18340881,Barbeque Nation,"G1, Villa, Near HQ Fitness, Near Lulu Mall, Ba...","Indian, North Indian",150,Emirati Diram(AED),Excellent,4.5
9167,11807,Barbeque Nation,"3rd Floor, R Deccan Mall, Near Deccan Gymkhana...","North Indian, Mughlai",1700,Indian Rupees(Rs.),Excellent,4.5


In [41]:
                                                            #same preferences which use initially
                                                        # Find all restaurants matching these preferences
user_preferences = {'Cuisines': 'Asian, Chinese, Vegetarian', 'City': 'Chandigarh', 'Rating': 2}
matching_restaurants = dt[
    (dt['City'] == user_preferences['City']) &
    (dt['Cuisines'].str.contains('|'.join(user_preferences['Cuisines']), case=False, na=False)) &
    (dt['Aggregate rating'] > user_preferences['Rating'])

]

# Recommend similar ones based on these preferences
recommendations = recommend_restaurants(matching_restaurants['Restaurant Name'].tolist(), num_recommendations=6)
recommendations = list(set(recommendations))
print("Recommended Restaurants:", recommendations)

Recommended Restaurants: ['The Culinary Pitaara', 'Barbeque Nation', 'Aromas of Pind']


In [42]:
dt[dt['Restaurant Name'].isin(recommendations)].sort_values(by='Aggregate rating',ascending=False).head(5)[['Restaurant ID','Restaurant Name','Address','Cuisines','Average Cost for two','Currency','Rating text','Aggregate rating']]

,Restaurant ID,Restaurant Name,Address,Cuisines,Average Cost for two,Currency,Rating text,Aggregate rating
9262,2800856,Barbeque Nation,"1st Floor, ATR Towers, Harbour Park Road, Pan...","North Indian, Chinese, Mediterranean",1600,Indian Rupees(Rs.),Excellent,4.9
2536,3300958,Barbeque Nation,"2nd Floor, Eternity Mall, Amravati Road, Sitab...","North Indian, European, Mediterranean",1600,Indian Rupees(Rs.),Excellent,4.9
2414,20842,Barbeque Nation,"K1, RDB Boulevard, Block EP & GP, Sector 5, Sa...","North Indian, Chinese",1600,Indian Rupees(Rs.),Excellent,4.9
2289,2100702,Barbeque Nation,"2nd Floor, Adityam Building, Ulubari, Guwahati",North Indian,1500,Indian Rupees(Rs.),Excellent,4.9
2409,25570,Barbeque Nation,"24, 1st Floor, Park Center Building, Park Stre...","North Indian, Chinese",1600,Indian Rupees(Rs.),Excellent,4.9
